# Фино настройване на Open AI модели

Този тетрадка е базирана на актуалните насоки, предоставени в [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) документацията на Open AI.

Финото настройване подобрява работата на базовите модели за вашето приложение, като ги дообучава с допълнителни данни и контекст, свързани с конкретния случай или сценарий. Имайте предвид, че техники като _few shot learning_ и _retrieval augmented generation_ ви позволяват да обогатите стандартния prompt с подходящи данни за по-добро качество. Тези подходи обаче са ограничени от максималния брой токени, които моделът може да обработи наведнъж.

С финото настройване на практика дообучаваме самия модел с нужните данни (което ни позволява да използваме много повече примери, отколкото могат да се поберат в максималния прозорец от токени) – и внедряваме _персонализирана_ версия на модела, която вече не изисква примери по време на инференция. Това не само подобрява ефективността на нашия prompt дизайн (имаме повече свобода да използваме токен прозореца за други неща), но потенциално и намалява разходите ни (като намалява броя токени, които трябва да изпратим към модела при инференция).

Финото настройване включва 4 стъпки:
1. Подгответе тренировъчните данни и ги качете.
1. Стартирайте обучението, за да получите фино настроен модел.
1. Оценете фино настроения модел и повторете за по-добро качество.
1. Внедрете фино настроения модел за инференция, когато сте доволни от резултатите.

Имайте предвид, че не всички базови модели поддържат фино настройване – [проверете документацията на OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) за най-актуална информация. Можете също да фино настроите вече фино настроен модел. В този урок ще използваме `gpt-35-turbo` като целеви базов модел за фино настройване.

---


### Стъпка 1.1: Подгответе вашия датасет

Нека създадем чатбот, който ви помага да разбирате периодичната таблица на елементите, като отговаря на въпроси за даден елемент с лимерик. В _този_ прост урок ще създадем датасет, с който да обучим модела, като включим няколко примерни отговора, които показват очаквания формат на данните. В реална ситуация ще трябва да създадете датасет с много повече примери. Може също да използвате отворен датасет (за вашата област), ако има такъв, и да го преформатирате за нуждите на финото обучение.

Тъй като се фокусираме върху `gpt-35-turbo` и искаме едноетапен отговор (chat completion), можем да създадем примери, използвайки [този препоръчан формат](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), който отразява изискванията на OpenAI за chat completion. Ако очаквате многoетапно разговорно съдържание, ще използвате [формата за многоетапни примери](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), който включва параметър `weight`, за да се посочи кои съобщения да се използват (или не) при финото обучение.

Тук ще използваме по-опростения едноетапен формат за нашия урок. Данните са във [jsonl формат](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) с по 1 запис на ред, като всеки е представен като JSON-обект. Примерът по-долу показва 2 записа – вижте [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) за пълния примерен набор (10 примера), който ще използваме за нашия урок по фино обучение. **Забележка:** Всеки запис _трябва_ да бъде дефиниран на един ред (да не се разделя на няколко реда, както е обичайно при форматиран JSON файл)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

В реална ситуация ще ви трябва много по-голям набор от примери за добри резултати – балансът е между качеството на отговорите и времето/разходите за фино обучение. Използваме малък набор, за да можем бързо да завършим финото обучение и да илюстрираме процеса. Вижте [този пример от OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) за по-сложен урок по фино обучение.


### Стъпка 1.2 Качете вашия набор от данни

Качете данните с помощта на Files API [както е описано тук](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Имайте предвид, че за да изпълните този код, първо трябва да сте направили следните стъпки:
 - Инсталирали сте Python пакета `openai` (уверете се, че използвате версия >=0.28.0 за най-новите функции)
 - Зададен е променливата на средата `OPENAI_API_KEY` с вашия OpenAI API ключ
За повече информация вижте [Ръководството за настройка](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), предоставено за курса.

Сега изпълнете кода, за да създадете файл за качване от вашия локален JSONL файл.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Стъпка 2.1: Създайте задача за фино настройване с помощта на SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Стъпка 2.2: Проверете статуса на задачата

Ето няколко неща, които можете да направите с API-то `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Показва последните n задачи за фино настройване
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Получавате подробности за конкретна задача за фино настройване
- `client.fine_tuning.jobs.cancel(<job_id>)` - Отменя задача за фино настройване
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Показва до n събития от задачата
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Първата стъпка в процеса е _валидиране на тренировъчния файл_, за да сте сигурни, че данните са в правилния формат.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Стъпка 2.3: Проследявайте събития, за да следите напредъка


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Стъпка 2.4: Вижте статуса в OpenAI таблото


Можете да видите статуса, като посетите уебсайта на OpenAI и разгледате секцията _Fine-tuning_ на платформата. Там ще видите статуса на текущата задача, както и ще можете да проследите историята на предишните изпълнения. На тази снимка се вижда, че предишното изпълнение е неуспешно, а второто е успешно. За контекст – това се случи, когато първото изпълнение използва JSON файл с неправилно форматирани записи – след като това беше поправено, второто изпълнение завърши успешно и моделът стана достъпен за използване.

![Статус на Fine-tuning задачата](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.bg.png)


Можете също така да видите съобщенията за състоянието и метриките, като превъртите надолу във визуалното табло, както е показано:

| Съобщения | Метрики |
|:---|:---|
| ![Съобщения](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.bg.png) |  ![Метрики](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.bg.png)|


### Стъпка 3.1: Вземете ID и тествайте фино настроения модел в кода


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Стъпка 3.2: Зареждане и тестване на фино-настроения модел в Playground

Вече можете да тествате фино-настроения модел по два начина. Първо, можете да посетите Playground и да използвате падащото меню Models, за да изберете новия си фино-настроен модел от наличните опции. Другият вариант е да използвате опцията "Playground", която се показва в панела за фино настройване (вижте снимката по-горе). Тя стартира _сравнителен_ изглед, който показва основния и фино-настроения модел един до друг за бърза оценка.

![Статус на фино-настройваща задача](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.bg.png)

Просто попълнете системния контекст, използван във вашите тренировъчни данни, и въведете тестовия си въпрос. Ще забележите, че и двете страни се обновяват с един и същ контекст и въпрос. Стартирайте сравнението и ще видите разликата в отговорите между тях. _Обърнете внимание как фино-настроеният модел форматира отговора според примера, който сте предоставили, докато основният модел просто следва системния prompt_.

![Статус на фино-настройваща задача](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.bg.png)

Ще забележите, че сравнението също така показва броя на токените за всеки модел и времето, необходимо за инференция. **Този конкретен пример е опростен и има за цел да покаже процеса, но не отразява реален набор от данни или ситуация**. Може да забележите, че и двата примера показват еднакъв брой токени (системният контекст и потребителският prompt са идентични), като фино-настроеният модел отнема повече време за инференция (персонализиран модел).

В реални ситуации няма да използвате толкова елементарен пример, а ще настройвате модела с реални данни (например продуктов каталог за обслужване на клиенти), където качеството на отговора ще бъде много по-очевидно. В _такъв_ контекст, за да постигнете същото качество на отговора с основния модел, ще е необходима по-сложна работа по създаване на prompt-и, което ще увеличи използването на токени и евентуално времето за обработка при инференция. _За да опитате това, разгледайте примерите за фино настройване в OpenAI Cookbook._



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, имайте предвид, че автоматичните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език следва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Не носим отговорност за недоразумения или погрешни тълкувания, произтичащи от използването на този превод.
